In [1]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2023-03-21 12:41:40--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip’

datasource.zip      100%[===================>]   4.15K  --.-KB/s    in 0s      

2023-03-21 12:41:41 (1013 MB/s) - ‘datasource.zip’ saved [4249/4249]



In [3]:
!unzip datasource.zip

Archive:  datasource.zip
  inflating: used_car_prices1.csv    
  inflating: used_car_prices2.csv    
  inflating: used_car_prices3.csv    
  inflating: used_car_prices1.json   
  inflating: used_car_prices2.json   
  inflating: used_car_prices3.json   
  inflating: used_car_prices1.xml    
  inflating: used_car_prices2.xml    
  inflating: used_car_prices3.xml    


In [12]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

In [13]:
# CSV extract function below
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

In [14]:
# JSON extract function below
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process, lines= True)
    return dataframe

In [15]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

In [16]:
# Round the price columns to 2 decimal places
def transform(data):
    data['price'] = round(data.price,2)
    return data

In [17]:
def load(target_file, data_to_load):
    data_to_load.to_csv(target_file)

In [18]:
def log(msg):
    timestamp_format = "%H:%M:%S-%h-%d-%Y"
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt", "a") as f:
        f.write(timestamp + "," + msg + "\n")

In [20]:
# Log that you have started the ETL process
log("ETL job start")

# Log that you have started the Extract step
log("Extract start")

# Call the Extract function
extracted_data = extract()

# Log that you have completed the Extract step
log("Extract End")

# Log that you have started the Transform step
log("Transform start")

# Call the Transform function
transformed_data = transform(extracted_data)
# Log that you have completed the Transform step
log("Transform_complete")

# Log that you have started the Load step
log("Load start")

# Call the Load function
load(targetfile, transformed_data)

# Log that you have completed the Load step
log("Load end")

# Log that you have completed the ETL process
log("ETL Complete")